## 1. 建立並保存 ONNX 模型檔案
以下是一個使用 TensorFlow 建立鳶尾花（Iris）分類模型並將其導出為 ONNX 格式的範例。該模型使用簡單的全連接層來進行分類，並轉換為 ONNX 格式，方便在 TVM 或其他 ONNX 支持的推理引擎上運行。

### 1.1 安裝必要的套件
如果尚未安裝 tensorflow 和 tf2onnx，可以使用以下命令安裝：



In [ ]:
!pip install tensorflow tf2onnx

### 1.2 建立並訓練 TensorFlow 模型
以下程式碼將建立一個簡單的神經網絡來分類鳶尾花數據集，並將其導出為 ONNX 格式。

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# 載入鳶尾花資料集
iris = load_iris()
X = iris.data.astype(np.float32)
y = iris.target

# 分割資料集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 建立模型
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(4,)),  # 4 個特徵
    tf.keras.layers.Dense(10, activation='relu'),  # 隱藏層
    tf.keras.layers.Dense(3, activation='softmax') # 輸出層，3 個分類
])

# 編譯模型
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 訓練模型
model.fit(X_train, y_train, epochs=50, batch_size=5, verbose=0)

# 評估模型
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"模型準確率: {accuracy:.2f}")

2024-11-12 20:53:47.762606: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-12 20:53:47.800341: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-12 20:53:47.800359: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-12 20:53:47.800385: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-12 20:53:47.807329: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-12 20:53:47.807845: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

模型準確率: 0.87


### 1.3 將模型轉換為 ONNX 格式
使用 tf2onnx 將訓練好的 TensorFlow 模型轉換為 ONNX 格式：

In [2]:
import tf2onnx

# 將 Keras 模型轉換為 ONNX 格式
spec = (tf.TensorSpec((None, 4), tf.float32, name="float_input"),)  # 定義輸入規範
output_path = "tf_model.onnx"  # 輸出 ONNX 模型的路徑

# 轉換模型
model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13)
with open(output_path, "wb") as f:
    f.write(model_proto.SerializeToString())

print(f"ONNX 模型已保存至 {output_path}")

ONNX 模型已保存至 tf_model.onnx


2024-11-12 20:54:01.888828: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-11-12 20:54:01.888952: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-11-12 20:54:01.911409: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-11-12 20:54:01.911518: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session


In [3]:
import onnxruntime as ort
import numpy as np

# 加載 ONNX 模型
session = ort.InferenceSession('tf_model.onnx')

# 準備輸入資料
input_name = session.get_inputs()[0].name
input_data = np.array([[6.3, 3.3, 6.0, 2.5]], dtype=np.float32)

# 進行推理
pred_onnx = session.run(None, {input_name: input_data})

# 輸出預測結果
print(pred_onnx)

[array([[6.0250495e-05, 2.8771785e-01, 7.1222192e-01]], dtype=float32)]


**環境設定與依賴安裝**
   - 在開發機器上安裝 TVM。
   - 在目標設備（aarch64）上安裝 TVM runtime。
   - 確保兩台設備在同一網絡下。

## 2. 模型匯入與轉換
將模型從原始格式（TensorFlow、PyTorch、ONNX）匯入並轉換為 TVM 的 Relay 格式。

In [1]:
import tvm
from tvm import rpc, relay
from tvm.contrib import utils
import onnx

# 載入 ONNX 模型
onnx_model = onnx.load("tf_model.onnx")

# 將 ONNX 模型轉換為 Relay 模型
input_name = 'float_input'  # 輸入名稱可在 ONNX 模型中確認
shape_dict = {input_name: (1, 4)}
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

## 3. 設定編譯目標並編譯模型
- 在開發機器上編譯模型，設定目標為 `"llvm -mtriple=aarch64-linux-gnu"`。
- 使用 TVM 自動調優進行性能優化（選擇性）。

In [2]:
# 在開發機器上編譯模型（例如為 aarch64 設備）
target = "llvm"
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, params=params)

One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


## 4. 在目標設備上啟動 RPC Server
在目標設備上運行 RPC Server，這允許開發機器通過 RPC 連接到目標設備並進行測試。

In [ ]:
# !python -m tvm.exec.rpc_server --host 0.0.0.0 --port=9090
# !ngrok tcp 0.0.0.0:9090

## 5. 模型部署與傳輸
- 將編譯後生成的二進制文件（例如 .tar 文件或 .so 文件）打包並傳輸到目標設備。
- 可以使用 TVM 的 RPC 機制來上傳文件，或手動傳輸文件到目標設備的指定目錄。

In [5]:
from tvm import rpc, relay
from tvm.contrib import utils

# 連線
remote = rpc.connect("0.tcp.jp.ngrok.io", 11685)

# 將編譯好的模型傳輸至目標設備
temp = utils.tempdir()
lib_fname = temp.relpath("deploy_lib.tar")
lib.export_library(lib_fname)
remote.upload(lib_fname)

# 在目標設備上加載模型
remote_lib = remote.load_module("deploy_lib.tar")
print(f'成功部署到{lib_fname}')
del remote
del remote_lib

成功部署到/tmp/tmp324x57mo/deploy_lib.tar


## 6. 連接目標設備並執行推論
- 在開發機器上連接到目標設備的 RPC Server，並加載已編譯的模型。
- 設置輸入數據並執行推論。


In [6]:
import numpy as np
import tvm
from tvm import rpc, relay
from tvm.contrib import utils, graph_executor

remote = rpc.connect("0.tcp.jp.ngrok.io", 11685)
# 在目標設備上加載模型
remote_lib = remote.load_module("/tmp/tmp324x57mo/deploy_lib.tar")

# 創建 graph executor，使用目標設備的 CPU
module = graph_executor.GraphModule(remote_lib["default"](remote.cpu()))

# 設置輸入數據
# 準備輸入資料
input_data = np.array([[6.3, 3.3, 6.0, 2.5]], dtype=np.float32)
# input_data = tvm.nd.array(input_array, device=remote.cpu())  # 輸入的數據
module.set_input("float_input", input_data)

# 執行推論
module.run()

# 取得輸出
output = module.get_output(0).asnumpy()
print("推論結果：", output)

# 釋放資源
del module
del remote_lib
del remote

推論結果： [[6.0250437e-05 2.8771785e-01 7.1222192e-01]]


In [8]:
!tar -xvf deploy_lib.tar

lib0.o
devc.o


## Reference
- [如何在TVM 中使用RPC 進行交叉編譯和遠端設備執行](https://tvm.hyper.ai/docs/tutorial/rpc/)